In [1]:
# Author: VISWA R
# Import and Get Cores
import multiprocessing as mp
import numpy as np
import tqdm
from itertools import repeat
from multiprocessing import Process, Manager
from multiprocessing import Pool
import pandas as pd


num_processes = mp.cpu_count()

print("Number of cpu : ", num_processes)

Number of cpu :  12


In [2]:
# Read Files
employee_df = pd.read_csv('employee.csv',low_memory=False)
data_df = pd.read_csv('data.csv',low_memory=False)

In [10]:
employee_df.head()

,ecode,ename
0,165,Harry Crouch
1,183,SIVANNA
2,190,JAYARAMAPPA
3,191,Sadiyah Mays
4,193,Macie Mccartney


In [3]:
# Convert Date ccolumn from String Type to date Type
data_df['date']= pd.to_datetime(data_df['date']) 

In [4]:
data_df.head()

,date,employee_code,efficiency
0,2018-01-01,82,86.445231
1,2018-01-01,101,107.360738
2,2018-01-01,1055,97.888896
3,2018-01-01,161,109.089957
4,2018-01-01,82,84.095728


In [15]:
# Get all Employee Codes from Employee Data
empcodes= []
empcodes.extend(list(data_df['employee_code'].unique()))
empcodes

[82,
 101,
 1055,
 161,
 1062,
 1083,
 1066,
 1071,
 1054,
 68,
 104,
 183,
 86,
 103,
 182,
 143,
 105,
 162,
 1065,
 145,
 250,
 245,
 165,
 240,
 146,
 230,
 236]

In [5]:
# Map Employee Codes with Name
name_dict = {}
for index,row in employee_df.iterrows():
    name_dict[row['ecode']] = row['ename']

In [18]:
name_dict

{'165': 'Harry Crouch',
 '183': 'SIVANNA',
 '190': 'JAYARAMAPPA',
 '191': 'Sadiyah Mays',
 '193': 'Macie Mccartney',
 '194': 'Adelaide Brook',
 '195': 'Shanon Mendez',
 '196': 'Isaak Burnett',
 '197': 'Susie Ramsey',
 '207': 'Sara Donnelly',
 '200': 'Emmanuel Waller',
 '103': 'Elif Noble',
 '122': 'DANEEL',
 '201': 'RAMACHANDRAREDDY',
 '203': 'Farah Caldwell',
 '210': 'Ashraf Rawlings',
 '211': 'Youssef Quintana',
 '238': 'RAMANJINAPPA',
 '215': 'Benn Kearney',
 '218': 'Mekhi Douglas',
 '220': 'Irene Jeffery',
 '250': 'Nathaniel Barron',
 '143': 'Kobi Hogan',
 '145': 'Sidney Ho',
 '1071': 'Wiktoria Wilkerson',
 '161': 'NARASHIMAHAMULU',
 '182': 'Teodor Greene',
 '249': 'Guto Bowden',
 '244': 'Ivy-Rose Chang',
 '68': 'Sneha Carr',
 '1054': 'Laurel Steele',
 '202': 'Bianca Naylor',
 '230': 'Nakita Miranda',
 '228': 'GUPTA',
 '1083': 'Arwel Eastwood',
 '86': 'Shiv Odling',
 '82': 'Patricia Ali',
 '233': 'Mehmet Mccaffrey',
 '206': 'Safiyah Salt',
 '189': 'Carlton Gunn',
 '184': 'Macey Row

In [23]:
# Do the MultiProcess
import workers
num_partitions = num_processes
manager = Manager()
d = manager.dict()
df_split = np.array_split(data_df, num_partitions)
pool = Pool(num_processes)
shared_arg = repeat(d,num_partitions)
for _ in tqdm.tqdm(pool.map(workers.process_rows, zip(shared_arg,df_split)), total=num_partitions):
    pass
pool.close()
pool.join()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<?, ?it/s]


In [24]:
# Geenrate Month Names
import datetime
months = []
for i in range(1,13):
    month = datetime.date(1900,i, 1).strftime('%B')
    months.append(month)

In [20]:
months

['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [25]:
# Plot Visualisation
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
data = []
for ecode in empcodes:
    y_list = []
    for i in range(1,13):
        y_list.append(d[int(ecode),i])
    data.append(go.Scatter(
    x=months,
    y=list(y_list),
    name = name_dict[str(ecode)]
    ))
layout = go.Layout(
    title='Employee Analysis',
    xaxis=dict(title='Months in 2018',),
    yaxis=dict(title='Average Production Efficieny per month',)
)

fig = go.Figure(data=data, layout=layout)
plot(fig, filename='Employee-Efficiency.html')

'Employee-Efficiency.html'